<a href="https://colab.research.google.com/github/nicolecosta/nlu/blob/main/IntRecV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prerequisites**

In [1]:
%pip install "tensorflow-text==2.8.*"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow_hub as hub
import numpy as np
from tensorflow_text import SentencepieceTokenizer
import pandas as pd
from sklearn.preprocessing import LabelEncoder

2022-10-05 23:11:47.371969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 23:11:47.372002: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import tensorflow as tf

In [4]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

2022-10-05 23:11:52.641302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 23:11:52.642986: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 23:11:52.643044: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-05 23:11:52.643085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-05 23:11:52.643125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
database = pd.read_csv("tfhubdatasett4.csv",sep=";")

print(database.shape)
database.head()

(16, 2)


,message,intent
0,oi,greet
1,"olá, tudo bem?",greet
2,"Oie, beleza?",greet
3,Eai,greet
4,Eu queria saber como tá o tempo hoje,weather


## **Data Augmentation**

In [6]:
import os
os.environ["MODEL_DIR"] = '../model'

In [7]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

/home/ubuntu/Projects/Labs/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 5 files to the new cache system


  0%|          | 0/5 [00:00<?, ?it/s]
There was a problem when trying to move your cache:

  File "/home/ubuntu/Projects/Labs/.venv/lib/python3.8/site-packages/transformers/utils/hub.py", line 1128, in <module>
    move_cache()

  File "/home/ubuntu/Projects/Labs/.venv/lib/python3.8/site-packages/transformers/utils/hub.py", line 1071, in move_cache
    hub_metadata[url] = get_hub_metadata(url, token=token)

  File "/home/ubuntu/Projects/Labs/.venv/lib/python3.8/site-packages/transformers/utils/hub.py", line 996, in get_hub_metadata
    huggingface_hub.file_download._raise_for_status(r)

AttributeError: module 'huggingface_hub.file_download' has no attribute '_raise_for_status'

Please file an issue at https://github.com/huggingface/transformers/issues/new/choose and copy paste this whole message and we will do our best to help.


### Keyboard Augmenter

In [2]:
text = database['message'][4]
aug = nac.KeyboardAug(aug_p=0.1)
augmented_text = aug.augment(text,n=3)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

NameError: name 'database' is not defined

### Random Augmenter

##### Substitute Character Randomly

In [9]:
aug = nac.RandomCharAug(action="swap")
augmented_text = aug.augment(text,n=3)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Eu queria saber como tá o tempo hoje
Augmented Text:
['Eu uqerai saber como tá o mtepo hoje', 'Eu queria saerb como tá o etpmo hoje', 'Eu qeuira saber omco tá o tepom hoje']


#### Delete Character Randomly

In [10]:
aug = nac.RandomCharAug(action="delete")
augmented_text = aug.augment(text, n=3)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Eu queria saber como tá o tempo hoje
Augmented Text:
['Eu uera saber como tá o emo oj', 'Eu queria ber como tá o teo oe', 'Eu ueia sab om tá o tempo hoje']


### Synonym Augmenter

In [58]:
aug = naw.SynonymAug(aug_src='wordnet',lang="por")
augmented_text = aug.augment(text, n=10)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Eu queria saber como tá o tempo hoje
Augmented Text:
['Eu queria saber configurado tá ó tempo hoje', 'Eu queria entender como tá ó tempo hoje', 'Eu queria conhecer como tá ó período hoje', 'Eu queria saber como tá ó escala de tempo geológico hoje', 'Eu queria saber como tá o estações (clima) hoje', 'Eu queria entender tomado tá o tempo actualidade', 'Eu queria saber tomado tá o fenómenos meteorológicos hoje', 'Eu queria conhecer tomado tá o tempo hoje', 'Eu queria conhecer como tá o estações do ano hoje', 'Eu queria entender tomado tá o tempo hoje']


### Contextual Word Embeddings Augmenter (BERT)

In [12]:
# Augment Portuguese hby BERT
aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', aug_p=0.1)
augmented_text = aug.augment(text,n=3)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
Eu queria saber como tá o tempo hoje
Augmented Text:
['eu queria saber como ta o que hoje', 'si queria saber como ta o tempo hoje', 'eu queria saber como seja o tempo hoje']


## **Making Database to Use Classifiers**

In [13]:
columnmessage = database['message']
embeddings = []
#tentar com Series
for message in columnmessage:
  #print(message)
  #msg = tf.constant(message)
  result = embed(message)
  result2 = result.numpy()[0]
  #results = np.array(result)
  #print(result)
  embeddings.append(result2)
  #embeddings.write(0,result)

#embeddings = np.array(embeddings)
#print(results)
print(embeddings)
#len(embeddings)
#len(embeddings)


[array([ 4.80121113e-02,  6.26051053e-02, -1.22022647e-02, -2.56003905e-02,
       -5.87953208e-03, -1.11547541e-02, -2.85057481e-02, -8.54070559e-02,
        6.96033007e-03, -2.32428852e-02, -2.36279275e-02, -5.55012142e-03,
        3.36342654e-03,  2.46568210e-02,  7.21149966e-02,  1.69200413e-02,
        5.69832474e-02,  9.86689515e-03, -5.23024052e-02,  1.42694796e-02,
        6.23171171e-03,  1.92785896e-02, -2.86019337e-03,  4.17078994e-02,
        4.58692107e-03, -4.08433937e-03,  2.08052322e-02, -5.51133137e-03,
        1.79002490e-02, -5.75575829e-02, -1.92000289e-02, -1.81767251e-02,
        1.29902223e-02, -8.68542269e-02, -2.68740989e-02, -6.02745637e-02,
       -5.56590632e-02, -2.18457822e-02, -2.55347602e-02, -2.85451729e-02,
        5.31172119e-02,  1.44329518e-02, -6.08275551e-03, -4.81291637e-02,
        1.11484546e-02, -6.92838896e-03,  2.54523773e-02,  4.26668599e-02,
        5.79169318e-02, -4.71465848e-02, -2.76831444e-02, -6.61670091e-03,
       -3.52222472e-02, 

In [14]:
# embeddings[0]

In [15]:
type(embeddings)

list

In [16]:
encoder = LabelEncoder()
intent = database['intent']
encodedintent = encoder.fit_transform(intent)
print(encodedintent)

[1 1 1 1 3 3 3 3 2 2 2 2 0 0 0 0]


In [17]:
data = {'EmbMessage': embeddings,
        'EncodIntent': encodedintent}
  
# Create DataFrame
df = pd.DataFrame(data)
df

,EmbMessage,EncodIntent
0,"[0.04801211, 0.062605105, -0.012202265, -0.025...",1
1,"[0.026949177, 0.01120004, -0.013866077, -0.094...",1
2,"[-0.02833372, 0.066245146, -0.020468066, -0.07...",1
3,"[0.07033987, 0.07651811, -0.019035531, -0.0228...",1
4,"[0.03175629, -0.011350906, -0.016366106, 0.047...",3
5,"[0.0664293, -0.052708466, -0.020370865, 0.0228...",3
6,"[0.029198188, -0.055035688, 0.029811844, 0.024...",3
7,"[0.0011842152, -0.06637811, 0.036279738, 0.019...",3
8,"[0.01160774, 0.008898978, -0.06061133, -0.0381...",2
9,"[-0.029394863, 0.016627867, -0.017552562, -0.0...",2


## **Decision Tree**

In [18]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    df,
    test_size=0.4,
    random_state=RANDOM_SEED,
)

In [19]:
train_set

,EmbMessage,EncodIntent
9,"[-0.029394863, 0.016627867, -0.017552562, -0.0...",2
2,"[-0.02833372, 0.066245146, -0.020468066, -0.07...",1
15,"[0.05327721, -0.045558725, 0.008196013, -0.064...",0
4,"[0.03175629, -0.011350906, -0.016366106, 0.047...",3
7,"[0.0011842152, -0.06637811, 0.036279738, 0.019...",3
10,"[0.03800388, -0.02746891, -0.03542512, -0.0064...",2
12,"[0.06322378, -0.022198105, -0.011628703, -0.06...",0
3,"[0.07033987, 0.07651811, -0.019035531, -0.0228...",1
6,"[0.029198188, -0.055035688, 0.029811844, 0.024...",3


In [20]:
x_train = train_set['EmbMessage']
y_train = train_set['EncodIntent']

In [21]:
x_test = test_set['EmbMessage']
y_test = test_set['EncodIntent']

In [22]:
x_train.to_list()

[array([-2.93948632e-02,  1.66278668e-02, -1.75525621e-02, -1.91692729e-02,
         1.20363226e-02, -4.78407815e-02, -3.92300673e-02, -5.50020412e-02,
         2.91033890e-02, -9.24798846e-02, -9.09107998e-02, -9.24988315e-02,
         2.83453111e-02,  5.36604831e-03,  5.14818579e-02,  4.29240391e-02,
        -7.73177072e-02,  5.85554317e-02,  3.09733748e-02,  3.06734666e-02,
         1.03011364e-02,  9.64933783e-02,  1.22035891e-02, -2.20251419e-02,
        -1.12325326e-02,  2.00006217e-02,  1.37744164e-02, -5.54012842e-02,
         3.78994271e-02, -3.57849412e-02,  4.59674783e-02,  3.21487710e-03,
         5.05264103e-02, -6.30819574e-02,  5.67789003e-02, -4.64999378e-02,
        -2.59783752e-02,  2.49479879e-02,  9.27758496e-03,  3.93989161e-02,
        -9.44993738e-03, -2.22427547e-02, -6.33950382e-02, -5.27887419e-02,
        -4.80618104e-02,  3.60849574e-02, -7.58344755e-02, -5.97054251e-02,
         2.91719977e-02, -2.26084609e-02, -5.35662845e-02, -7.84946457e-02,
         1.3

In [23]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(x_train.to_list(), y_train)

DecisionTreeClassifier()

In [24]:
from sklearn.metrics import accuracy_score

y_pred = tree_clf.predict(x_test.to_list())
dt_accuracy = accuracy_score(y_test, y_pred)
print('DecisionTreeClassifier accuracy score: {}'.format(dt_accuracy))


DecisionTreeClassifier accuracy score: 0.42857142857142855


In [25]:
#teste de plot da decision tree

# def plot_tree(tree, dataframe, label_col, plot_title):
#     label_names = pd.unique(dataframe[label_col])
#     # Obtaining plot data.
#     graph_data = export_graphviz(tree,
#     feature_names=dataframe.drop(label_col, axis=1).columns,
#     class_names=label_names,
#     filled=True,
#     rounded=True,
#     out_file=None)
#     # Generating plot.
#     graph = graphviz.Source(graph_data)
#     graph.render(plot_title)
#     return graph

# tree_graph = plot_tree(tree_clf, df, 'EncodIntent', 'NLU Tree')
# tree_graph


## **Random Forest**

In [26]:
from sklearn.ensemble import RandomForestClassifier

# cria modelo
model = RandomForestClassifier(random_state = 0)

# treina modelo
model.fit(x_train.to_list(), y_train)

RandomForestClassifier(random_state=0)

In [27]:
# Testa modelo
y_predict = model.predict(x_test.to_list())
# Cálcula acurácia
rf_accuracy = accuracy_score(y_test, y_predict)
print("Acurácia: {0}".format(acc))

NameError: name 'acc' is not defined

## **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

# cria modelo
modelr = LogisticRegression(solver='lbfgs', multi_class='auto')
modelr.fit(x_train.to_list(), y_train)

LogisticRegression()

In [ ]:
y_pred_lr = modelr.predict(x_test.to_list())
acc_LogisticRegression = accuracy_score(y_test, y_pred_lr)
print("Acurácia: {0}".format(acc_LogisticRegression))

Acurácia: 0.8571428571428571


## **Result Comparison**

In [ ]:
print('Decision Tree:',dt_accuracy)
print('Logistic Regression:',acc_LogisticRegression)
print('Random Forest Classifier:',rf_accuracy)

Decision Tree: 0.42857142857142855
Logistic Regression: 0.8571428571428571
Random Forest Classifier: 0.7142857142857143
